In [26]:
from scipy.optimize import minimize
import numpy as np

In [27]:
# error function parameters  modes
eta = 9.180582707595432
'''rho = pm.Uniform("rho", lower=0, upper=10)
beta_L = pm.Uniform("beta_L", lower=0, upper=3)
beta_H = pm.Uniform("beta_H", lower=0, upper=6)'''
rho = 0.2043253342838957
beta_L = 1.1851555212967242
beta_H = 0.6769375801601656
# rho_2 = pm.Uniform("rho_2", lower=0, upper=1)
beta_H2 = 1.9693465099568148
# alpha_1 = pm.Normal("alpha_1", mu=10, sigma=5)
alpha_L = 10.336437586660763
alpha_1 = 9.148634792876067
alpha_2 = 20.68057204828696

sigma_lg_Phi = 0.09505097667012101

# computational cost
C_L = .994 + 1.845  # C_L1 + C_L2 in CPU hours
C_H = 178.813

q = C_H / C_L

# objective error
err_0 = .008 # goal error


In [28]:
def err_func(n_LF, n_HF):
    Phi = eta * (rho / ( (n_LF + n_HF) * (n_LF + alpha_L)**(beta_L-1) * (n_HF + alpha_1)**(beta_H-1) ) + 1 / (alpha_2 + n_HF)**(beta_H2))
    # median_Phi = 10**lg_Phi
    return Phi

def err_mean(n_LF, n_HF):
    gamma = np.exp(1/2 * (sigma_lg_Phi * np.log(10))**2)
    return gamma * err_func(n_LF, n_HF)

def comp_bud(n_L, n_H):
    return C_L * n_L + C_H * n_H

In [29]:


# # Define the objective function
def objective(vars):
    n_L, n_H = vars
    return comp_bud(n_L, n_H)

# Define the constraint function
def constraint(vars):
    n_L, n_H = vars
    return err_mean(n_L,n_H) - err_0

# Initial guess for the variables
initial_guess = [500, 20]

# Define the constraint dictionary
con = {'type': 'eq', 'fun': constraint}

# Perform the optimization
result = minimize(objective, initial_guess, constraints=[con])

# Extract the optimal values of x and y
x_opt, y_opt = result.x

print(f"Optimal values: x = {x_opt}, y = {y_opt}")
print(f"Function value at optimal points: f(x, y) = {objective((x_opt, y_opt))}")

Optimal values: x = 716.706223907556, y = 22.51139377663635
Function value at optimal points: f(x, y) = 6060.058825055226


In [30]:
comp_bud(x_opt, y_opt) / 56

108.21533616170048

In [31]:
comp_bud(x_opt, y_opt) / 56 * 1200

129858.40339404058

In [32]:
n_LF = 564
n_HF = 21

In [33]:
err_mean(n_LF,n_HF)

0.00911231797132653

In [34]:
comp_bud(n_LF, n_HF) / 56 * 1200

114777.19285714286

In [35]:
err_mean(0, 66.3)

0.07743380559941093

In [36]:
comp_bud(0, 66.3) / 56 * 1200

254042.18357142853

In [37]:
err_func(10000, 0)

0.023624056310636233

In [38]:
# error function at integer n_LF and n_HF
n_LF = 78
n_HF = 9
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 78, n_HF = 9: 208.52%
C at n_LF = 78, n_HF = 9: 1830.759 CPU hours = 32.69 node hours
